In [1]:
import pandas as pd

df = pd.read_csv("../data/movies.csv")

df.columns

Index(['movie_name', 'year', 'runtime', 'genre', 'rating', 'description',
       'director', 'star', 'votes'],
      dtype='object')

In [3]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [4]:
df["name_descr_genre_director_star"] = df.apply(lambda row: f'{row.movie_name} {row.description} {row.genre} {row.director} {row.star}', axis=1)

df["name_descr_genre_director_star_vector"] = embedding_model.encode(df["name_descr_genre_director_star"], show_progress_bar=True, device="mps:0").tolist()

df.drop("name_descr_genre_director_star", axis=1, inplace=True)

Batches:   0%|          | 0/6988 [00:00<?, ?it/s]

In [5]:
df["movie_name_vector"] = embedding_model.encode(df["movie_name"], show_progress_bar=True, device="mps:0").tolist()
df["description_vector"] = embedding_model.encode(df["description"], show_progress_bar=True, device="mps:0").tolist()

Batches:   0%|          | 0/6988 [00:00<?, ?it/s]

Batches:   0%|          | 0/6988 [00:00<?, ?it/s]

In [10]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "movie_name": {"type": "text"},
            "year": {"type": "text"},
            "runtime": {"type": "text"},
            "genre": {"type": "text"},
            "rating": {"type": "text"},
            "description": {"type": "text"},
            "director": {"type": "text"},
            "star": {"type": "text"},
            "votes": {"type": "text"},
            "id": {"type": "keyword"},
            "movie_name_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "description_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "name_descr_genre_director_star_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "movies-database"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies-database'})

In [11]:
from tqdm import tqdm

for i, doc in tqdm(df.iterrows(), total=len(df)):
    es_client.index(index=index_name, document=doc.to_json())

100%|██████████| 223589/223589 [17:18<00:00, 215.21it/s]


In [47]:
def elastic_search_knn(question, vector, num_results):
    knn = {
        "field": "name_descr_genre_director_star_vector",
        "query_vector": vector,
        "k": num_results,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ['movie_name', 'year', 'runtime', 'genre', 'rating',
            'description', 'director', 'star', 'votes', 'id']
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query,
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
        result_docs[-1]['score'] = hit['_score']


    return result_docs

In [162]:
question = "How many Avatar movies they are and what are they about?"
question_vector = embedding_model.encode(question)
for doc in elastic_search_knn(question, question_vector, 10):
    print(doc["movie_name"])

Avatar 4
Avatar 5
Avatar
Avatar 3
Rifftrax: Avatar
Untitled Avatar: The Last Airbender Film 1
Avatara Purusha: Part 1
Xbox Avatar Fun
Digimon: The Movie
Zoo Wars 2


In [14]:
prompt = """
You are a movie recommendation assistant. You must answer the QUESTION based on the CONTEXT. If you cannot find the answer to the question, just say "I don't know", don't try to make up an answer. Do not make up an answer if you cannot find the answer to the question. Don't mention the source of the information. Avoid using the words context and dataset.

QUESTION:
{question}

CONTEXT: 
{context}
"""

In [48]:
def get_reponse_openai(client, model_name, search_func):
    res = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt.format(
            question=question,
            context = search_func(question, question_vector, 10))}],
    )
    print(res.choices[0].message.content)

In [17]:
from openai import OpenAI

client_llama = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama"
)

get_reponse_openai(client_llama, "llama3.1", elastic_search_knn)

There are multiple movies with the title "Avatar" in them, but I will provide an answer based on the question and context.

To answer your question: There are four Avatar movies mentioned in the provided list:

1. The original movie "Avatar" (2009) - a sci-fi epic directed by James Cameron.
2. "Avatar 3" (2024) - a sequel to "Avatar: The Way of Water", with the plot unknown.
3. "Avatar 4" (2026) - described as a sequel to "Avatar 3" and also with an unknown plot, and presumably third installment in a planned trilogy of sequels for the 2009 original film.
4. Although not explicitly called “Avatar 5” there is mention of a last movie of the saga and therefore it can be assumed that there will indeed be one more Avatar movie after Avatar 4:


In [19]:
gpt_client = OpenAI()

In [20]:
get_reponse_openai(gpt_client, "gpt-4o", elastic_search_knn)

There are currently five Avatar movies:

1. **Avatar (2009)**: This film introduces the world of Pandora, where a paraplegic Marine is dispatched on a unique mission and finds himself torn between following his orders and protecting the world that feels like home.

2. **Avatar: The Way of Water (2022)**: Although not mentioned in the provided information, this is the known sequel to the first Avatar movie.

3. **Avatar 3 (2024)**: This film is the sequel to Avatar: The Way of Water. The specific plot details are unknown.

4. **Avatar 4 (2026)**: This film continues the story from Avatar 3. The plot is unknown.

5. **Avatar 5 (2028)**: This film is the sequel to Avatar 4 and is projected to be the last movie in the Avatar saga. The plot is unknown.


In [49]:
get_reponse_openai(gpt_client, "gpt-4o-mini", elastic_search_knn)

There are five Avatar movies planned. 

1. **Avatar (2009)** - A paraplegic Marine is dispatched to the moon Pandora on a unique mission, where he becomes torn between following his orders and protecting the world he feels is his home.

2. **Avatar: The Way of Water (2022)** - The plot details are not provided in the context.

3. **Avatar 3 (2024)** - This is a sequel to Avatar: The Way of Water, but the plot is unknown.

4. **Avatar 4 (2026)** - A sequel to Avatar 3, with the plot also unknown.

5. **Avatar 5 (2028)** - The final movie of the Avatar saga and a sequel to Avatar 4, with the plot not revealed.

The first film is the only one with a specific plot description available. The subsequent films have their plots unknown.


## Github Models

I've gained recently access to models hosted by Github in their beta of Github Models. Let's see how selected models will behave.
A link to the models list: https://github.com/marketplace/models

In [50]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_response_azure_github(model_name, search_func):
    response = client.complete(
        messages=[
            UserMessage(content=prompt.format(
            question=question,
            context = search_func(question, question_vector, 10)),),
        ],
        model=model_name,
        temperature=1.0,
        max_tokens=1000,
        top_p=1.0
    )   
    print(response.choices[0].message.content)

In [32]:
model_name = "meta-llama-3.1-405b-instruct"

get_response_azure_github(model_name, elastic_search_knn)

There are 5 Avatar movies directed by James Cameron: 

1. Avatar (2009) - A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.
2. Avatar 3 (2024) - The plot is unknown, but it is a sequel to Avatar: The Way of Water (2022).
3. Avatar 4 (2026) - The plot is unknown, but it is a sequel to Avatar 3 (2024).
4. Avatar 5 (2028) - The plot is unknown, but it is a sequel to Avatar 4 (2026) and the last movie of the "Avatar" saga.

Note: Avatar: The Way of Water (2022) is mentioned as a predecessor to Avatar 3, but it is not included in the provided data.


In [33]:
model_name = "meta-llama-3.1-70b-instruct"

get_response_azure_github(model_name, elastic_search_knn)

There are five Avatar movies. Here's a brief description of each:

1. Avatar (2009) - A paraplegic Marine becomes torn between following orders and protecting the world he feels is his home on the moon Pandora.

2. Avatar: The Way of Water (2022) - No plot description available.

3. Avatar 3 (2024) - No plot description available.

4. Avatar 4 (2026) - No plot description available.

5. Avatar 5 (2028) - No plot description available. It's also the last movie in the "Avatar" saga.


In [34]:
model_name = "Mistral-nemo"

get_response_azure_github(model_name, elastic_search_knn)

There are two Avatar movies:
1. "Avatar" (2009): A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home. Rated 7.9.
2. "Avatar: The Way of Water" (2022):Unknown; sequel to the first Avatar


In [35]:
model_name = "Mistral-large"

get_response_azure_github(model_name, elastic_search_knn)

There are four Avatar movies in the context provided, although not all of them have been released yet. Here's a brief overview:

1. "Avatar" (2009): This is the first movie in the series. It's about a paraplegic Marine who is dispatched to the moon Pandora on a unique mission. He becomes torn between following his orders and protecting the world he feels is his home.

2. "Avatar 3" (2024): This is a sequel to "Avatar: The Way of Water" (which isn't detailed in the context). The plot for "Avatar 3" is currently unknown.

3. "Avatar 4" (2026): This is a sequel to "Avatar 3". The plot is also unknown.

4. "Avatar 5" (2028): This is the final movie in the "Avatar" saga and a sequel to "Avatar 4". The plot is yet to be revealed.

While there are other movies with "Avatar" in the title, they are not part of this series.


In [36]:
model_name = "Mistral-large-2407"

get_response_azure_github(model_name, elastic_search_knn)

There are 5 "Avatar" movies in the series directed by James Cameron:

1. **Avatar (2009):** A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.
2. **Avatar 3 (2024):** Sequel of Avatar: The Way of Water (2022). The plot is unknown.
3. **Avatar 4 (2026):** Sequel of Avatar 3 (2024). The plot is unknown.
4. **Avatar 5 (2028):** Sequel of Avatar 4 (2026) and last movie of the "Avatar" saga. The plot is unknown.

Note: "Avatar: The Way of Water" (2022) is mentioned as the sequel to the original "Avatar" (2009), but its details are not provided in the list.


In [37]:
model_name = "Phi-3-medium-128k-instruct"

get_response_azure_github(model_name, elastic_search_knn)

 There are five Avatar movies. Avatar (2009) is about a paraplegic Marine who is torn between his orders and protecting a world he considers home. Avatar: The Way of Water (not listed explicitly but can be inferred was released in 2022) is a sequel of the lot which follows the events set up in the first movie. Avatar 3 (2024) is another sequel following the plot from the previous films. Avatar 4 (2026) and Avatar 5 (2028) are sequels as well, with the plot for each being unknown at this time.


### Conclusion

We can see that the best behaving and cheapest model is GPT 4o-mini. Let's stick to it.
llama3.1:8b is pretty poor, although totally free and run via ollama. We can see that llama3.1:70b is much better but, for reproducibility sake, and given that Github Models are in closed beta.

## Rephrase question

Let's try to change the query that we are searching with elastic search. We could optimize it. Let's use gpt 4o-mini for this.

In [119]:
def rephrase_query(question):
    query = f"""
        Given a QUESTION, extract one or more words from the QUESTION that can indicate one or more of 
        following fields: movie_name, year, runtime, genre, rating, description,
        director, star, votes.
        Output the extracted words separated by a space.

        QUESTION
        {question}
    """
    res = gpt_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": query}],
    )
    return res.choices[0].message.content

In [120]:
rephrased_question = rephrase_query(question)

rephrased_question_vector = embedding_model.encode(rephrased_question)

In [124]:
for doc in elastic_search_knn(rephrased_question, rephrased_question_vector, 10):
    print(doc["movie_name"])

Avatar
Avatar
The Avatar Project
Avatar 4
Xbox Avatar Fun
Rifftrax: Avatar
Untitled Avatar: The Last Airbender Film 1
Avatar 3
Avatar 5
The King's Avatar: For the Glory


In [188]:
def get_reponse_gpt4omini(question, search_func, num_results=10):
    rephrased_question = rephrase_query(question)
    rephrased_question_vector = embedding_model.encode(rephrased_question)
    res = gpt_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt.format(
            question=question,
            context = search_func(rephrased_question, rephrased_question_vector, num_results))}]
    )
    print(res.choices[0].message.content)

In [130]:
get_reponse_gpt4omini(question, elastic_search_knn)

There are multiple Avatar movies, specifically:

1. **Avatar (2009)** - A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.

2. **Avatar 3 (2024)** - Sequel of Avatar: The Way of Water (2022). The plot is unknown.

3. **Avatar 4 (2026)** - Sequel of Avatar 3. The plot is unknown.

4. **Avatar 5 (2028)** - Sequel of Avatar 4 and the last movie of the "Avatar" saga. The plot is unknown.

In total, there are four known films in the series, with the plots of the later three films not specified.


Given the search results, the output is a bit better, however still lacks Avatar 2 entry. Let's experiment further with elastic search to check if we can get it to appear.

In [131]:
def elastic_search_knn_more_vectors(question, vector, num_results):
    knn = [
        {
            "field": "name_descr_genre_director_star_vector",
            "query_vector": vector,
            "k": num_results,
            "num_candidates": 10000,
            "boost": 0.9
        },
        {
            "field": "movie_name_vector",
            "query_vector": vector,
            "k": num_results,
            "num_candidates": 10000,
            "boost": 0.9
        },
        {
            "field": "description_vector",
            "query_vector": vector,
            "k": num_results,
            "num_candidates": 10000,
            "boost": 0.9
        }
    ]

    search_body = {
        "knn": knn,
        "_source": [
            'movie_name', 'year', 'runtime', 'genre', 'rating',
            'description', 'director', 'star', 'votes', 'id'
        ]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_body
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
        result_docs[-1]['score'] = hit['_score']

    return result_docs

In [133]:
for doc in elastic_search_knn_more_vectors(rephrased_question, rephrased_question_vector, 10):
    print(doc["movie_name"])

Avatar
Avatar
Avatar 4
The Avatar Project
Avatar 5
Avatar 3
Rifftrax: Avatar
Xbox Avatar Fun
Avatar
Avataran


In [132]:
get_reponse_gpt4omini(question, elastic_search_knn_more_vectors)

There are four main Avatar movies. 

1. **Avatar (2009)** - A paraplegic Marine is sent to the moon Pandora on a mission but finds himself torn between his orders and protecting the world he feels is his home.

2. **Avatar: The Way of Water (2022)** - Not detailed in the provided context, but it's a sequel to the original Avatar.

3. **Avatar 3 (2024)** - A sequel to "Avatar: The Way of Water" with an unknown plot.

4. **Avatar 4 (2026)** - A sequel to "Avatar 3" with an unknown plot.

5. **Avatar 5 (2028)** - The last movie of the "Avatar" saga, also with an unknown plot.

Additionally, there are other films with the title Avatar, but they are not part of the main series.


In [167]:
def elastic_search_knn_query(question, vector, num_results):
    knn = {
        "field": "name_descr_genre_director_star_vector",
        "query_vector": vector,
        "k": 60,
        "num_candidates": 10000,
        "boost": 0.5
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question,
                    "fields": ['movie_name', 'description', 'director', 'star', 'year', 'runtime', 'genre', 'rating'],
                    "type": "best_fields",
                    "boost": 0.7,
                }
            }
        }
    }

    search_query = {
        "knn": knn,
        "query": keyword_query,
        "_source": ['movie_name', 'year', 'runtime', 'genre', 'rating',
            'description', 'director', 'star', 'votes', 'id']
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query,
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits'][:num_results]:
        result_docs.append(hit['_source'])
        result_docs[-1]['score'] = hit['_score']


    return result_docs

In [170]:
for res in elastic_search_knn_query(rephrased_question, rephrased_question_vector, 10):
    print(res["movie_name"])

Avatar 5
Avatar
Avatar
Avatar 4
Avatar
Avatar 3
Rifftrax: Avatar
Three Tutus and a Gun
Sita Swayamvar
A Perfect World


In [144]:
get_reponse_gpt4omini(question, elastic_search_knn_query)

There are five Avatar movies planned:

1. **Avatar (2009)**: This film follows a paraplegic Marine who is dispatched to the moon Pandora on a unique mission. He becomes torn between following his orders and protecting the world he feels is his home.

2. **Avatar: The Way of Water (2022)**: The plot details are not provided in the context.

3. **Avatar 3 (2024)**: This is a sequel to *Avatar: The Way of Water*, but the plot is unknown.

4. **Avatar 4 (2026)**: This is a sequel to *Avatar 3*, and the plot is also unknown.

5. **Avatar 5 (2028)**: This movie is the sequel to *Avatar 4* and is described as the last film of the "Avatar" saga, with the plot currently unknown.

Additionally, there are other films titled "Avatar" from different years, but they are not part of the main Avatar saga.


In [176]:
def compute_rrf(rank, k):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(query, vector, num_results):
    k=600
    knn_query = {
        "field": "name_descr_genre_director_star_vector",
        "query_vector": vector,
        "k": 60,
        "num_candidates": 10000,
        "boost": 0.7,
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ['movie_name', 'description', 'director', 'star', 'year', 'runtime', 'genre', 'rating'],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            }
        }
    }

    knn_results = es_client.search(
        index=index_name, 
        body={
            "knn": knn_query, 
            "size": 10
        }
    )['hits']['hits']
    
    keyword_results = es_client.search(
        index=index_name, 
        body={
            "query": keyword_query, 
            "size": 10
        }
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:num_results]:
        doc = es_client.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])
    
    return final_results

In [191]:
for res in elastic_search_hybrid_rrf(rephrased_question, rephrased_question_vector, 6):
    print(res["movie_name"])

Avatar
Avatar
Avatar 4
Avatar 5
Rifftrax: Avatar
Avatar 3


In [192]:
get_reponse_gpt4omini(question, elastic_search_hybrid_rrf, 6)

There are four "Avatar" movies to date:

1. **Avatar (2009)** - This is the original movie where a paraplegic Marine, Jake Sully, is sent to the moon Pandora on a unique mission. As he becomes emotionally attached to the world and its people, he is torn between following his orders and protecting the native Na'vi people.

2. **Avatar: The Way of Water (2022)** - The sequel explores new environments and the struggle of the Sully family, as well as the challenges they face from external threats.

3. **Avatar 3 (2024)** - This sequel continues the saga but the plot details are currently unknown.

4. **Avatar 4 (2026)** - This film is a sequel to Avatar 3, and its plot details are still unknown.

Additionally, there are plans for a fifth installment, **Avatar 5 (2028)**, which will be the final movie in the series, but specific details about its plot have not been disclosed yet.
